# **Colab notebook for StitchingNet-Seg benchmark**
- Hyungjung Kim revises this Colab notebook from the original Python codes composed byJunhyeok Park.
- The notebook was executed using a CUDA-enabled GPU environment.
- Before running the code, you should upload StitchingNet-Seg.zip to the content folder.



## **Part 1. Dataset preparation**

In [1]:
import os
import cv2
import torch
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Configuration
IMAGE_ROOT = "/content/StitchingNet-Seg/dataset"
IMAGESETS_DIR = "/content/StitchingNet-Seg/splits"
EXCLUDED_CLASSES = [6, 8, 9] # from StitchingNet
IMG_EXTENSIONS = ['.jpg', '.png']

# Mapping from dataset folder ID (Defect Type) to segmentation class ID
# Format: {Folder_ID: Segmentation_class_ID}
SEG_CLASS_MAP = {1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 7: 6, 10: 7} # total 8 classes
NUM_CLASSES = len(SEG_CLASS_MAP) + 1  # +1 for Background

In [2]:
def get_file_paths():
    """
    Traverses the dataset directory to collect image and mask paths.
    Returns lists of metadata for train, validation, and test sets.
    """
    fabric_folders = sorted([d for d in os.listdir(IMAGE_ROOT) if os.path.isdir(os.path.join(IMAGE_ROOT, d))])

    all_image_meta = {}

    for fabric_folder in fabric_folders:
        fabric_path = os.path.join(IMAGE_ROOT, fabric_folder)

        for class_folder in os.listdir(fabric_path):
            try:
                # Folder name indicates the defective class ID
                class_id = int(class_folder.split('.')[0])
            except ValueError:
                continue

            if class_id in EXCLUDED_CLASSES:
                continue

            class_path_base = os.path.join(fabric_path, class_folder)
            image_dir = os.path.join(class_path_base, 'image')
            mask_dir = os.path.join(class_path_base, 'mask')

            if not os.path.isdir(image_dir) or not os.path.isdir(mask_dir):
                continue

            for filename in os.listdir(image_dir):
                basename, ext = os.path.splitext(filename)
                if ext.lower() not in IMG_EXTENSIONS:
                    continue

                img_path = os.path.join(image_dir, filename)
                mask_path = os.path.join(mask_dir, basename + '.png')

                if not os.path.exists(mask_path):
                    continue

                # Store metadata: (image_path, mask_path, defective_class_id)
                all_image_meta[basename] = (img_path, mask_path, class_id)

    def load_data_from_split_file(split_filename):
        """
        Loads file paths based on the text files in the splits directory.
        """
        split_path = os.path.join(IMAGESETS_DIR, split_filename)
        if not os.path.exists(split_path):
            raise FileNotFoundError(f"Split file not found: {split_path}")

        data_list = []
        with open(split_path, 'r') as f:
            basenames = [line.strip() for line in f if line.strip()]

        for basename in basenames:
            meta = all_image_meta.get(basename)
            if meta:
                data_list.append(meta)

        return data_list

    train_data = load_data_from_split_file("train.txt")
    val_data = load_data_from_split_file("val.txt")
    test_data = load_data_from_split_file("test.txt")

    return train_data, val_data, test_data

In [3]:
class SegmentationDataset(Dataset):
    """
    Customize dataset for semantic segmentation.
    """
    def __init__(self, file_data, seg_map, transforms=None):
        self.file_data = file_data
        self.seg_map = seg_map
        self.transforms = transforms

    def __len__(self):
        return len(self.file_data)

    def __getitem__(self, idx):
        img_path, mask_path, original_defect_id = self.file_data[idx]

        # Load image
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Load mask
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            raise IOError(f"Failed to read mask file: {mask_path}")

        # Generate segmentation mask based on defect ID
        seg_mask = np.zeros_like(mask, dtype=np.int64)

        if original_defect_id != 0:
            target_seg_id = self.seg_map[original_defect_id]
            seg_mask[mask > 0] = target_seg_id

        if self.transforms:
            transformed = self.transforms(image=image, masks=[seg_mask])
            image = transformed['image']
            seg_mask = transformed['masks'][0]

        return image, seg_mask.long()

In [4]:
# Utility functions
def get_transforms(image_size=256):
    """
    Returns Albumentations transforms for training and validation.
    """
    train_transform = A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])
    val_test_transform = A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ])
    return train_transform, val_test_transform

def seed_worker(worker_id):
    """
    Ensures reproducibility in data loading.
    """
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def get_dataloaders(batch_size=8, image_size=256, num_workers=4, seed=42):
    """
    Creates and returns DataLoaders for train, val, and test sets.
    """
    train_data, val_data, test_data = get_file_paths()
    train_transform, val_test_transform = get_transforms(image_size)

    train_dataset = SegmentationDataset(train_data, SEG_CLASS_MAP, transforms=train_transform)
    val_dataset = SegmentationDataset(val_data, SEG_CLASS_MAP, transforms=val_test_transform)
    test_dataset = SegmentationDataset(test_data, SEG_CLASS_MAP, transforms=val_test_transform)

    g = torch.Generator()
    g.manual_seed(seed)

    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=True,
        worker_init_fn=seed_worker, generator=g
    )
    val_loader = DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True,
        worker_init_fn=seed_worker
    )
    test_loader = DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=True,
        worker_init_fn=seed_worker
    )

    return train_loader, val_loader, test_loader, NUM_CLASSES

In [5]:
!unzip /content/StitchingNet-Seg.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_078.png  
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_079.png  
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_080.png  
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_081.png  
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_082.png  
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_083.png  
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_084.png  
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_085.png  
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_086.png  
  inflating: StitchingNet-Seg/dataset/I. Jacquard-Poly/2. Broken stitch/mask/I_02_087.png  
  inflating: StitchingNet-Seg/dataset/I. Jac

In [6]:
print("Initializing DataLoaDers...")
train_dl, val_dl, test_dl, num_classes = get_dataloaders(batch_size=4, num_workers=0)

print(f"Number of segmentation classes: {num_classes}")
print(f"Train batches: {len(train_dl)}")

if len(train_dl) > 0:
    print("\nChecking first batch...")
    images, seg_masks = next(iter(train_dl))

    print(f"Image shape: {images.shape}")
    print(f"Mask shape: {seg_masks.shape}")
    print(f"Unique mask values: {torch.unique(seg_masks)}")
else:
    print("\nError: Train dataset is empty.")

Initializing DataLoaDers...
Number of segmentation classes: 8
Train batches: 1626

Checking first batch...
Image shape: torch.Size([4, 3, 256, 256])
Mask shape: torch.Size([4, 256, 256])
Unique mask values: tensor([0])


## **Part 2. Run benchmark**

In [7]:
!pip install segmentation_models_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 4.9 MB/s eta 0:00:00


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import segmentation_models_pytorch as smp
from tqdm import tqdm
import numpy as np
import os
import sys
import pandas as pd
import random
import matplotlib.pyplot as plt

# Configuration & hyperparameters
BASE_DIR = "./result_models"
sys.path.append(BASE_DIR)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EPOCHS = 100
LEARNING_RATE = 1e-4
BATCH_SIZE = 16
IMAGE_SIZE = 224
PATIENCE = 8
SEED = 64
NUM_WORKERS = 4

# Directory setup
MODEL_SAVE_DIR = os.path.join(BASE_DIR, "trained_models")
RESULT_SAVE_DIR = os.path.join(BASE_DIR, "benchmark_results")
VIS_SAVE_DIR = os.path.join(BASE_DIR, "visual_results")

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(RESULT_SAVE_DIR, exist_ok=True)
os.makedirs(VIS_SAVE_DIR, exist_ok=True)

In [9]:
# Color map for visualization (fixed colors for consistency)
# Format: [R, G, B]
COLOR_MAP = [
    [0, 0, 0],       # Background
    [255, 0, 0],     # Class 1
    [0, 255, 0],     # Class 2
    [0, 0, 255],     # Class 3
    [255, 255, 0],   # Class 4
    [255, 0, 255],   # Class 5
    [0, 255, 255],   # Class 6
    [128, 0, 0],     # Class 7
    [0, 128, 0],     # Class 8
    [0, 0, 128],     # Class 9
]

In [10]:
# Model
class ModelFactory(nn.Module):
    """
    Factory class to instantiate various segmentation models using SMP.
    """
    def __init__(self, model_name, num_classes):
        super().__init__()
        self.model_name = model_name

        if model_name == 'ResUNet':
            self.model = smp.Unet(
                encoder_name="resnet34", encoder_weights="imagenet",
                in_channels=3, classes=num_classes
            )
        elif model_name == 'UNet++':
            self.model = smp.UnetPlusPlus(
                encoder_name="resnet34", encoder_weights="imagenet",
                in_channels=3, classes=num_classes
            )
        elif model_name == 'DeepLabV3':
            self.model = smp.DeepLabV3Plus(
                encoder_name="resnet34", encoder_weights="imagenet",
                in_channels=3, classes=num_classes
            )
        elif model_name == 'SegFormer':
            self.model = smp.Segformer(
                encoder_name="mit_b2", encoder_weights="imagenet",
                in_channels=3, classes=num_classes
            )
        elif model_name == 'Swin-Unet':
            self.model = smp.Unet(
                encoder_name="tu-swin_tiny_patch4_window7_224", encoder_weights="imagenet",
                in_channels=3, classes=num_classes
            )
        else:
            raise ValueError(f"Unknown model name: {model_name}")

    def forward(self, x):
            return self.model(x)

In [11]:
# Utility functions
def set_seed(seed=42):
    """
    Sets random seeds for reproducibility.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    os.environ['PYTHONHASHSEED'] = str(seed)

def _fast_hist(label, pred, n_class):
    """
    Calculates the confusion matrix for segmentation.
    """
    mask = (label >= 0) & (label < n_class)
    hist = np.bincount(
        n_class * label[mask].astype(int) + pred[mask],
        minlength=n_class ** 2
    ).reshape(n_class, n_class)
    return hist

def calculate_metrics(hist):
    """
    Computes mIoU, mDice, and Recall from the confusion matrix.
    """
    epsilon = 1e-7
    tp = np.diag(hist)
    fp = hist.sum(axis=0) - tp
    fn = hist.sum(axis=1) - tp

    iou = tp / (tp + fp + fn + epsilon)
    dice = 2 * tp / (2 * tp + fp + fn + epsilon)
    recall = tp / (tp + fn + epsilon)

    miou = np.nanmean(iou)
    mdice = np.nanmean(dice)
    mrecall = np.nanmean(recall)

    return miou, mdice, mrecall

def colorize_mask(mask):
    """
    Converts a (H, W) label mask into a (H, W, 3) RGB image for visualization.
    """
    if isinstance(mask, torch.Tensor):
        mask = mask.cpu().numpy()

    h, w = mask.shape
    img = np.zeros((h, w, 3), dtype=np.uint8)

    for class_id, color in enumerate(COLOR_MAP):
        img[mask == class_id] = color

    return img

def denormalize(tensor, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    """
    Denormalizes a tensor image back to the [0, 255] range.
    """
    img = tensor.cpu().numpy().transpose(1, 2, 0)
    img = (img * std + mean)
    img = np.clip(img, 0, 1)
    return (img * 255).astype(np.uint8)

def save_visual_results(model_name, images, gt_masks, pred_masks, batch_idx, save_dir):
    """
    Saves a comparison of Original, GT, and Prediction images.
    """
    os.makedirs(save_dir, exist_ok=True)
    batch_size = images.size(0)

    for i in range(batch_size):
        orig_img = denormalize(images[i])
        gt_color = colorize_mask(gt_masks[i])
        pred_color = colorize_mask(pred_masks[i])
        fig, ax = plt.subplots(1, 3, figsize=(12, 4))

        ax[0].imshow(orig_img)
        ax[0].set_title(f"Original")
        ax[0].axis('off')

        ax[1].imshow(gt_color)
        ax[1].set_title("Ground Truth")
        ax[1].axis('off')

        ax[2].imshow(pred_color)
        ax[2].set_title(f"{model_name} Prediction")
        ax[2].axis('off')

        file_name = f"batch{batch_idx:02d}_img{i:03d}_{model_name}.png"

        save_path = os.path.join(save_dir, file_name)
        plt.tight_layout()
        plt.savefig(save_path)
        plt.close(fig)

In [19]:
# Training & evaluation
def run_experiment(model_name, train_dl, val_dl, test_dl, num_classes):
    print(f"\n" + "="*60)
    print(f"Start Training: {model_name}")
    print("="*60)

    model = ModelFactory(model_name, num_classes).to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)

    # Loss functions
    loss_dice = smp.losses.DiceLoss(mode='multiclass')
    loss_ce = nn.CrossEntropyLoss()

    best_miou = 0.0
    save_path = os.path.join(MODEL_SAVE_DIR, f"{model_name}_best.pth")
    epochs_no_improve = 0

    # Training
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
        loop = tqdm(train_dl, desc=f"[{model_name}] Ep {epoch+1}/{EPOCHS}", leave=False)

        for imgs, masks in loop:
            imgs, masks = imgs.to(DEVICE), masks.to(DEVICE, dtype=torch.long)

            optimizer.zero_grad()
            outputs = model(imgs)

            # Interpolate if output size mismatch
            if outputs.shape[-2:] != masks.shape[-2:]:
                outputs = nn.functional.interpolate(outputs, size=masks.shape[-2:], mode='bilinear', align_corners=False)

            loss = loss_dice(outputs, masks) + loss_ce(outputs, masks)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        # Validation
        model.eval()
        hist = np.zeros((num_classes, num_classes))

        with torch.no_grad():
            for imgs, masks in val_dl:
                imgs = imgs.to(DEVICE)
                masks = masks.numpy()

                outputs = model(imgs)
                if outputs.shape[-2:] != imgs.shape[-2:]:
                    outputs = nn.functional.interpolate(outputs, size=imgs.shape[-2:], mode='bilinear', align_corners=False)

                preds = torch.argmax(outputs, dim=1).cpu().numpy()
                hist += _fast_hist(masks.flatten(), preds.flatten(), num_classes)

        miou, mdice, _ = calculate_metrics(hist)
        print(f"[Ep {epoch+1}] Val -> Train_loss: {train_loss/len(train_dl):.4f} | mIoU: {miou:.4f} | mDice: {mdice:.4f}")

        # Save best model
        if miou > best_miou:
            best_miou = miou
            torch.save(model.state_dict(), save_path)
            epochs_no_improve = 0
            print(f"   -> Best model saved! (mIoU: {best_miou:.4f})")
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= PATIENCE:
                print(f"   -> Early stopping triggered at epoch {epoch+1}.")
                break

    # Final benchmark (load best model & visualize)
    print(f"\nEvaluating best {model_name} on the test dataset...")
    model.load_state_dict(torch.load(save_path))
    model.eval()

    model_vis_dir = os.path.join(VIS_SAVE_DIR, model_name)
    os.makedirs(model_vis_dir, exist_ok=True)

    hist = np.zeros((num_classes, num_classes))
    save_count = 0
    MAX_SAVE_BATCHES = 5

    with torch.no_grad():
        for batch_idx, (imgs, masks) in enumerate(tqdm(test_dl, desc="Testing")):
            imgs = imgs.to(DEVICE)

            outputs = model(imgs)
            if outputs.shape[-2:] != imgs.shape[-2:]:
                outputs = nn.functional.interpolate(outputs, size=imgs.shape[-2:], mode='bilinear', align_corners=False)

            preds = torch.argmax(outputs, dim=1)

            gt_numpy = masks.numpy()
            pred_numpy = preds.cpu().numpy()
            hist += _fast_hist(gt_numpy.flatten(), pred_numpy.flatten(), num_classes)

            if save_count < MAX_SAVE_BATCHES:
                save_visual_results(model_name, imgs, masks, preds, batch_idx, model_vis_dir)
                save_count += 1

    test_miou, test_mdice, test_recall = calculate_metrics(hist)
    print(f"Final benchmark result ({model_name}) -> mIoU: {test_miou:.4f}, Dice: {test_mdice:.4f}")
    print(f"Visual results saved to: {model_vis_dir}")

    return {
        'Model': model_name,
        'mIoU': test_miou,
        'mDice': test_mdice,
        'Recall': test_recall
    }

In [21]:
# Run benchmark
set_seed(SEED)
print(f"The experiment for model benchmark is started on {DEVICE}")
print(f"Base directory: {BASE_DIR}")

# 1. Load StitchingNet-Seg dataset
train_dl, val_dl, test_dl, num_classes = get_dataloaders(
    batch_size=BATCH_SIZE, image_size=IMAGE_SIZE, num_workers=NUM_WORKERS, seed=SEED
)

# 2. Set target models to benchmark
# target_models = ['ResUNet', 'UNet++', 'DeepLabV3', 'SegFormer', 'Swin-Unet']
target_models = ['ResUNet', 'SegFormer']
results = []

# 3. Run experiments
for model_name in target_models:
    set_seed(SEED)
    try:
        res = run_experiment(model_name, train_dl, val_dl, test_dl, num_classes)
        results.append(res)
    except Exception as e:
        print(f"Error training {model_name}: {e}")
        import traceback
        traceback.print_exc()

# 4. Save final benchmark results
if results:
    df = pd.DataFrame(results)
    df = df.sort_values(by='mIoU', ascending=False)

    save_path = os.path.join(RESULT_SAVE_DIR, "final_benchmark_results.csv")
    df.to_csv(save_path, index=False)

    print("\n" + "="*60)
    print("Final benchmark results")
    print("="*60)
    print(df.to_string(index=False))
    print(f"\nResults saved to: {save_path}")
else:
    print("No results to save.")

The experiment for model benchmark is started on cuda
Base directory: ./result_models

Start Training: ResUNet


[Ep 1] Val -> Train_loss: 1.3469 | mIoU: 0.1233 | mDice: 0.1244
   -> Best model saved! (mIoU: 0.1233)


[Ep 2] Val -> Train_loss: 0.7964 | mIoU: 0.2170 | mDice: 0.2563
   -> Best model saved! (mIoU: 0.2170)


[Ep 3] Val -> Train_loss: 0.6486 | mIoU: 0.4006 | mDice: 0.5179
   -> Best model saved! (mIoU: 0.4006)


[Ep 4] Val -> Train_loss: 0.4532 | mIoU: 0.4765 | mDice: 0.6017
   -> Best model saved! (mIoU: 0.4765)


...

[Ep 39] Val -> Train_loss: 0.0856 | mIoU: 0.7455 | mDice: 0.8501


[Ep 40] Val -> Train_loss: 0.0852 | mIoU: 0.7461 | mDice: 0.8504


[Ep 41] Val -> Train_loss: 0.0830 | mIoU: 0.7466 | mDice: 0.8507
   -> Early stopping triggered at epoch 41.

Evaluating best SegFormer on the test dataset...


Testing: 100%|██████████| 136/136 [00:29<00:00,  4.61it/s]

Final benchmark result (SegFormer) -> mIoU: 0.7444, Dice: 0.8489
Visual results saved to: ./result_models/visual_results/SegFormer

Final benchmark results
    Model     mIoU    mDice   Recall
SegFormer 0.744449 0.848896 0.861105
  ResUNet 0.737312 0.844326 0.835530

Results saved to: ./result_models/benchmark_results/final_benchmark_results.csv
